# Bayesian Modeling

### MAP Estimation
Point estimates from a posterior distribution:
- mean
- median
- mode (== the MAP estimate)

The mean and median are usually the more appropriate choices, but the mode (== the MAP estimate) is very popular.

Pros:
- this reduces to an optimization problem
- the prior can be interpreted as a regularizer

Cons:
- the mode is usually not a very representative point estimate of the distribution
- the mode estimate is sensitive to re-parameterization

## Bayesian Model Selection
Bayesian model selection is an alternative to K-fold cross-validation, in which the selected model (parametrized by $\theta$) is that which maximizes the posterior, given the data:

$$
\begin{align}
    \hat{m} &= argmax_m[p(m|D)] \\
            &= argmax_m[\frac {p(D|m)p(m)} {\sum_{\tilde{m} \in M} p(D|\tilde{m}) p(\tilde{m})}] \\
\end{align}
$$

Since $\sum_{\tilde{m} \in M} p(D|\tilde{m}) p(\tilde{m})$ is a normalization constant, it does not affect the solution to the $argmax$. Thus we are left with

$$
\begin{align}
    \hat{m} &= argmax_m[p(m|D)] \\
            &= argmax_m[p(D|m)p(m)]
\end{align}
$$

If, for now, we assume a uniform prior, this solution simplifies to maximizing $p(D|m)$. But where are the parameters $\theta$? In this expression, they have been integrated out. We can use the total probability rule to bring $\theta$ back into the expression:

$$
\begin{align}
    \hat{m} &= argmax_m[p(m|D)] \\
            &= argmax_m[p(D|m)] \\
            &= argmax_m[\int p(D|\theta, m) p(\theta|m) d\theta]
\end{align}
$$

The integral goes by several names:

1. Integrated likelihood
2. Marginal likelihood
3. Evidence for model $m$

## Bayesian Model Selection Follows Occam's Razor
Note how each element in the integration above performs its own distinct form of model selection:
* $p(D|\theta)$
    * (approximation) correlates with how much the data "makes sense" in reference to the parameters $\theta$
* $p(\theta|m)$
    * (complexity regularization) correlates with how much the parameters $\theta$ "make sense" in reference to the model $m$
* $p(m)$
    * (a priori regularization) correlates with our biased beliefs about the universe

## Computing the Marginal Likelihood

How do we compute the marginal data likelihood $p(D|m)$? No problem! By the total probability rule, we can use the integral above: $\int p(D|\theta)p(\theta|m) d \theta.$

Wait...that's an unbounded integration over **_all_** possible parameters $\theta$. That might take a while...

Once again, it's conjugate priors to the rescue! Assume that for any $x$, $p(x) = \frac {q(x)} {Z_x}$ is a normalized probability distribution, where the unnormalized distribution $q(x)$ is divided by the appropriate normalization constant $Z_x$. Therefore, for (respectively) a prior, likelihood, and posterior $p(\theta), p(D|\theta), p(\theta|D)$:

$$
\begin{align}
    p(\theta) &= \frac {q(\theta)} {Z_0} \\
    p(D|\theta) &= \frac {q(D|\theta)} {Z_l} \\
    p(\theta|D) &= \frac {q(\theta|D)} {Z_N}
\end{align}
$$

Now, we can equate this representation of $p(\theta|D)$ with that of Baye's rule, and substitute in each "normalization representation" of those terms:

$$
\begin{align}
    p(\theta|D) &= \frac {q(\theta|D)} {Z_N} = \frac {p(\theta)p(D|\theta)} {p(D)} \\
                &= \frac {q(\theta|D)} {Z_N} = \frac {\frac {q(\theta)} {Z_0}\frac {q(D|\theta)} {Z_l}} {p(D)} \\
                &= q(\theta|D) = \frac {\frac {Z_N} {Z_0 Z_l} q(\theta)q(D|\theta)} {p(D)}
\end{align}
$$

Now, note that , since the $q(x)$ are unnormalized quantities, $q(\theta|D) = q(\theta)q(D|\theta)$. Therefore,

$$
\begin{align}
       1 &= \frac {\frac {Z_N} {Z_0 Z_l}} {p(D)} \\
 => p(D) &= \frac {Z_N} {Z_0 Z_l}
\end{align}
$$

#### Computing the Marginal Likelihood: Beta-binomial

Recall that, for the beta-binomial model, we have the following Bayesian breakdown:

$$
\begin{align}
    p(\theta) &= Beta(\theta|a,b) \\
              &\propto \theta^{a-1}(1-\theta)^{b-1} \\\\
  p(D|\theta) &= Bin(N_1|N_1+N_0,\theta) \\
              &= \binom N {N_1} \theta^{N_1} {(1-\theta)}^{1-N_1} \\\\
  p(\theta|D) &= Beta(\theta|N_1+a,N_0+b) \\
              &= \frac 1 {B(N_1+a,N_0+b)} x^{N_1+a-1} {(1-x)}^{N_0+b-1} \\
              &\ \ \ \ \,  B(a,b) = \frac {\Gamma(a) \Gamma(b)} {\Gamma(a, b)}
\end{align}
$$

Since we know that the posterior follows a Beta distribution, we immediately know the form of **its** normalization constant: $B(N_1+a,N_0+b).$ Plugging this into the definition of the posterior $p(\theta|D)$, we can solve for the marginal likelihood $p(D)$:

$$
\begin{align}
    p(D) &= \frac {Beta(\theta|a,b)Bin(N_1|N_1+N_0,\theta)} {Beta(\theta|N_1+a,N_0+b)} \\
          &= \binom N {N_1} \frac {B(N_1+a,N_0+b)} {B(a,b)}
\end{align}
$$

#### Computing the Marginal Likelihood: Beta-Bernoulli

Recalling that the Bernoulli distribution is just a special case of the Binomial distribution (where the event only occurs ones), the marginal likelihood for the model is similar to that for the Beta-Binomial model, with the binomial term $\binom N N_1$ removed:

$$p(D) = \frac {B(N_1+a,N_0+b)} {B(a,b)}$$

#### Computing the Marginal Likelihood: Dirichlet-Multinoulli

Recalling that the Multinoulli distribution is just a multivariate generalization of the Bernoulli distribution, the marginal likelihood for the model is similar to that for the Beta-Binomial model, with the Beta function in its multivariate form:

$$
\begin{align}
    B(\alpha) &= \frac {\prod_{k=1}^K \Gamma(\alpha_k)} {\Gamma(\sum_k \alpha_k)} \\
         p(D) &= \frac {B(N+\alpha)} {B(\alpha)} \\
              &= \frac {\Gamma(\sum_k \alpha_k)} {\Gamma(N+\sum_k \alpha_k)} \prod_k \frac {\Gamma(N_k+\alpha_k)} {\Gamma(\alpha_k)} \\
\end{align}
$$

#### Computing the Marginal Likelihood: Gaussian-Gaussian-Wishart

$$
\begin{align}
    p(D) &= \frac {1} {\pi^{\frac {ND} 2}} (\frac {\kappa_0} {\kappa_N})^{\frac D 2} \frac {|{S_0}|^{\nu_0/2}} {|{S_N}|^{\nu_N / 2}} \frac {\Gamma_D(\nu_N/2)} {\Gamma_D(\nu_0/2)}
\end{align}
$$

<br/><br/>
## Marginal Likelihood Approximations

A popular approach to approximating the marginal likelihood $p(D)$ when it is intractable to compute directly is to estimate the _log_ marginal likelihood $log(p(D))$ with one of two information criteria, shown below.

#### Marginal Likelihood Approximations: Bayesian Information Criterion (BIC)

In this approach, we assume that $log(p(D)) \approx BIC$, where

$$
\begin{align}
    BIC &= log\ p(D|\hat{\theta}) - \frac {dof(\hat{\theta})} 2 log\ N \\
        &= log\ likelihood - complexity\ penalty
\end{align}
$$

Depending on context, either the ML or MAP estimate can be used for $\hat{\theta}$.

## Empirical Bayes (Type-II Maximum Likelihood)
http://varianceexplained.org/r/empirical_bayes_baseball/
TODO notes



#### Example Gaussian-Gaussian Model of Student Scores Across Schools (Uniform Variance Among Schools)
Let $x_{ij}$ be the score for the $i^{th}$ student ($N$ total) in the j^{th} school ($D$ total). We want to compute the mean test score _per-school_, but face a problem: class imbalance. How should we approach the fact that the schools have an un-even amount of students? One approach is to approach the problem _hierarchically_, and assume that the per-school score means $\theta_j$ come from a common prior, $N(\mu,\tau^2)$. This gives us the following joint distribution of the scores and mean estimates:

$$
\begin{align}
                         \eta &= \{\mu,\tau^2\} \\
    p(\theta,D|\eta,\sigma^2) &= p(\theta|\eta)\ p(D|\theta, \sigma^2_j) \\
                              &= \prod_{j=1}^D N(\theta_j|\mu,\tau^2) \prod_{i=1}^{N_j} N(x_{ij}|\theta_j,\sigma^2_j)
\end{align}
$$

For the moment, let's assume (1) that we have successfully estimated the hyperparameters $\hat{\eta} = \{\hat{\mu}, \hat{\tau}^2\}$. Also note an important property (2): $N$ Gaussian measurements with sample mean $\bar{\mu_j}$ and sample variance $\bar{\sigma}_j^2$ is _equivalent_ to a single Gaussian measurement with mean $\bar{\mu_j}$ and variance $\bar{\sigma}^2_j$. Using the aforementioned assumption (1) and property (2), we can re-write the joint distribution in terms of the single Gaussian measurement:

$$
\begin{align}
    p(\theta,D|\hat{\eta},\sigma^2) &= \prod_{j=1}^D N(\theta_j|\hat{\mu},\hat{\tau}^2) N(\bar{x_{j}}|\theta_j,\bar{\sigma}^2_j) \\
                                    &= N(\theta_j|mean,var) \\
                                    &\ \ \ \ \ mean = \hat{S}_j\hat{\mu} + (1-\hat{S}_j)\bar{x}_j \\
                                    &\ \ \ \ \ var = (1-\hat{S}_j)\sigma^2_j \\
                                    &\ \ \ \ \ \hat{S}_j = \frac {\sigma^2_j} {\sigma^2_j + \hat{\tau}^2} \\
\end{align}
$$

The key control here are the $0 \le \hat{S}_j \le 1$. They control the _shrinkage_ of our estimates towards the _overall_ mean $\mu$. Note the behavior in the following two cases of sample size $N_j$ for the $j^th$ school:

|             | Variance Comparison          | $\hat{S}_j size$  | Mean Favored | Overall Variance             |
| ----------- | ---------------------------- | ----------------- | ------------ | ---------------------------- |
| $N_j$ large | $\sigma_j \ll \hat{\tau}^2$  | small             | $\bar{x}_j$  | $var \Rightarrow 0$          | 
| $N_j$ small | $\sigma_j \gg \hat{\tau}^2$  | large             | $\hat{\mu}$  | $var \Rightarrow \sigma^2_j$ |


Now, let's actually estimate the hyperparameters $\eta = \{\mu,\tau^2\}$. The distribution of school means is

$$
\begin{align}
                        p(\bar{x_j}) &= \int P(\bar{x_j} \cap \theta_j)d\theta_j \\
    p(\bar{x_j}|\mu,\tau^2,\sigma^2) &= \int N(\bar{x_j}|\theta_j,\sigma^2) N(\theta_j|\mu,\tau^2)d\theta_j \\
    p(\bar{x_j}|\mu,\tau^2,\sigma^2) &= N(\bar{x_j}|\mu, \sigma^2+\tau^2)
\end{align}
$$

The marginal likelihood of the data $D$ is

$$
\begin{align}
                        p(D) &= \prod_{j=1}^D p(\bar{x_j}) \\
    p(D|\mu,\tau^2,\sigma^2) &= \prod_{j=1}^D p(\bar{x_j}|\mu,\tau^2,\sigma^2) \\
\end{align}
$$

So we can use the MLE for $\mu$, which in turn is the overall mean:
$$
\begin{align}
    \hat{\mu} &= \frac 1 D \sum_{j=1}^D \bar{x_j} \\
              &= \bar{x}
\end{align}
$$

The variance estimate is obtained using moment matching, which is equivalent to the MLE for Gaussians. In moment matching, the model's variance is set equal to the empirical variance $s^2$, and then the equality is solved for the desired variance estimate $\tau^2$. Since, in this case, the model's variance is simply $\tau^2 + \sigma^2$, the variance estimate is
$$\hat{\tau}^2 = s^2 - \sigma^2$$
$$s^2 = \frac 1 D \sum_{j=1}^D {(\bar{x_j} - \bar{x})}^2$$

With these estimates in hand, the final shrinkage quantity of the model is:
$$
\begin{align}
    \hat{S} &= \frac {\sigma^2} {\sigma^2 + \hat{\tau}^2} \\
            &= \frac {\sigma^2} {\sigma^2 + {(s^2 - \sigma^2)}^2} 
\end{align}
$$

#### Example Gaussian-Gaussian Model of Student Scores Across Schools (Non-Uniform Variance Among Schools)
TODO using EM (Exercise 11.13)